In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from awesome.run.awesome_config import AwesomeConfig
from awesome.run.awesome_runner import AwesomeRunner
from awesome.util.reflection import class_name
from awesome.analytics.result_model import ResultModel
from awesome.util.path_tools import get_project_root_path, get_package_root_path
from awesome.util.logging import basic_config
import os
import torch
import numpy as np
import re
from awesome.util.format import latex_postprocessor
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import to_hex
from colour import Color
basic_config()

os.chdir(get_project_root_path()) # Beeing in the root directory of the project is important for the relative paths to work consistently

## Teaser Implicit Figure

In [ ]:
path = "./notebooks/data/TEST_CNNET_benchmark+xy+convex_23_09_28_11_34_39"

result_model = ResultModel.from_path(path)

In [ ]:
runner = result_model.get_runner(-1)

In [ ]:
from awesome.run.functions import get_result, plot_image_scribbles
model = runner.agent._get_model()
dataloader = runner.dataloader

index = 0


In [ ]:
i_0 = list(model.state_dict().items())[0]

s_0 = list(result_model.checkpoint_index.iloc[0]["checkpoint"].model_state_dict.items())[0]

(i_0[1] == s_0[1]).all()

In [ ]:
from awesome.run.functions import get_result, plot_image_scribbles, split_model_result
from awesome.util.temporary_property import TemporaryProperty

index = 0
res, ground_truth, img, fg, bg = get_result(model, dataloader, index, model_gets_targets=False)
res = split_model_result(res, model, dataloader, img, compute_crf=False)

with TemporaryProperty(model, use_prior_sigmoid=False):
    _wo_s = get_result(model, dataloader, index, model_gets_targets=False)
    res_hull = _wo_s[0][0, 1, ...][None, ...]

import numpy as np
image = img
x = np.arange(0, image.shape[2], 1)
y = np.arange(0, image.shape[1], 1)
xx, yy = np.meshgrid(x, y)


color_fg = plt.get_cmap("tab10").colors[2]
color_bg = plt.get_cmap("tab10").colors[3]

color_seg = plt.get_cmap("tab10").colors[0]

fig = plot_image_scribbles(img, res.get("segmentation"), fg, bg, res.get("prior"), size=10, tight_layout=True, tight=True)
display(fig)
plt.close(fig)

In [ ]:
import numpy as np
image = img
x = np.arange(0, image.shape[2], 1)
y = np.arange(0, image.shape[1], 1)
xx, yy = np.meshgrid(x, y)


color_fg = plt.get_cmap("tab10").colors[2]
color_bg = plt.get_cmap("tab10").colors[3]

color_seg = plt.get_cmap("tab10").colors[0]

In [ ]:

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb

import numpy as np

def colorFader(start,end,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    start=np.array(mpl.colors.to_rgb(start))
    end=np.array(mpl.colors.to_rgb(end))
    return mpl.colors.to_hex((1-mix)*start + mix*end)

def gradient_end_transparent_map(start_color, end_color, name: str, alpha: float = 1) -> str:
    start_color = to_hex(start_color)

    end_color = to_hex(end_color)

    #start_color = Color(start_color)
    # get colormap
    ncolors = 256
    #colors = list([x.get_rgb() for x in start_color.range_to(Color(end_color), ncolors)])
    colors = [to_rgb(colorFader(start_color, end_color, x / (ncolors-1))) for x in range(ncolors)]
    color_array = np.array(colors)
    
    rgba_array = np.ones(color_array.shape[:1] + (4,))
    rgba_array[..., :3] = color_array
    
    #color_array[..., -1] = 1.

    # First and last should be fully transparent
    rgba_array[0, -1] = 0
    rgba_array[1, :3] = rgba_array[0, :3]


    rgba_array[-1, -1] = 0

    rgba_array[-2, :3] = rgba_array[-1, :3]

    rgba_array[:, -1] = rgba_array[:, -1] * alpha

    # create a colormap object
    map_object = LinearSegmentedColormap.from_list(colors=rgba_array, name=name)
    return map_object

In [ ]:
from matplotlib.axes import Axes


gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
def plot_color_gradients(cmap_category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=nrows, figsize=(6.4, figh))
    fig.subplots_adjust(top=1-.35/figh, bottom=.15/figh, left=0.2, right=0.99)

    if isinstance(axs, Axes):
        axs = [axs]

    axs[0].set_title(f"{cmap_category} colormaps", fontsize=14)

    for ax, cmap_name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=cmap_name)
        ax.text(-.01, .5, cmap_name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

cmap = gradient_end_transparent_map(color_fg, color_bg, "FG-BG")

plot_color_gradients("Color", [cmap])

In [ ]:
print(to_hex(color_bg))
print(to_hex(to_hex(cmap(254))))

In [ ]:

from typing import Optional, Any
from matplotlib.axes import Axes
from awesome.run.functions import saveable, plot_surface_logits
    
plot_surface_logits(image, res_hull, fg, bg, color_fg=color_fg, color_bg=color_bg, save=True, override=True, transparent=True, 
                    path="./notebooks/output/convex_teaser/teaser_implicit", 
                    ext=["png", "pdf"], zoom=1.3)

In [ ]:
from awesome.run.functions import register_alpha_colormap

@saveable()
def plot_scribble_mask(img, fg, bg, color_fg, color_bg):
    register_alpha_colormap(color_fg, "tab10_green_alpha")
    register_alpha_colormap(color_bg, "tab10_red_alpha")

    size = 10
    tight = True

    sizes = image.shape[1:]
    fig = plt.figure(figsize=(size, size))
    dpi = 300
    fig.set_size_inches(size * (sizes[1] / dpi), size * (sizes[0] / dpi), forward=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)


    ax.imshow(img.permute(1, 2, 0).numpy())


    fgh = ax.imshow(fg, cmap='tab10_green_alpha')
    bgh = ax.imshow(bg, cmap='tab10_red_alpha')

    ax.axis('off')
    return fig
plot_scribble_mask(img, fg, bg, color_fg, color_bg, save=True, override=True, transparent=True, path="./notebooks/output/convex_teaser/teaser_scribbles", ext=["png", "pdf"])

In [ ]:
# Multi images
import math
import numpy as np

num_images = 24 * 6
rng = 2 * math.pi

base_azimuth = 270.
base_elev = 30.

max_elev = 60
min_elev = 0

total_elev_dist = (max_elev - base_elev) + (max_elev - min_elev) + (base_elev - min_elev)
elev_step = total_elev_dist / num_images
elevations = np.array(np.arange(base_elev, max_elev, elev_step).tolist() + np.arange(max_elev, min_elev, -elev_step).tolist() + np.arange(min_elev, base_elev, elev_step).tolist())

steps = np.arange(-np.pi, np.pi, (2 * np.pi / num_images))
def rad_to_deg(x):
    return x * 180 / math.pi

def minmax(v: torch.Tensor, new_min: float = 0., new_max: float = 1.):
    v_min, v_max = v.min(), v.max()
    return (v - v_min)/(v_max - v_min)*(new_max - new_min) + new_min

azimuts = (base_azimuth + rad_to_deg(steps) + 180) % 360
base_dir = "./notebooks/output/gif2"
os.makedirs(base_dir, exist_ok=True)
for i in range(num_images):
    frac_height = None
    elevation = elevations[i]
    azimuth = azimuts[i]
    with plt.ioff():
        plot_surface_logits(image, res_hull, fg, bg, 
                            color_fg=color_fg, 
                            color_bg=color_bg, 
                            save=True, 
                            override=True, 
                            path=os.path.join(base_dir, f"teaser_implicit_{i}"), 
                            ext=["png"],
                            elevation=elevation,
                            azimuth=azimuth,
                            zoom=1
                            )
        plt.close()

In [ ]:
from awesome.util.gif_writer_images import GifWriterImages

import re
result_path = "./notebooks/output/gif2/"

pattern_file = r"teaser_implicit_(?P<iter>\d+).png"

images = []

for img in os.listdir(result_path):
    m = re.fullmatch(pattern_file, img)
    if m:
        idx = int(m.group("iter"))
        images.append((idx, os.path.join(result_path, img)))
        continue

images = [x[1] for x in sorted(images, key=lambda x: x[0])]

#images = 
gifw = GifWriterImages("teaser_implicit_animation.gif", images)



In [ ]:
gifw.temp_dir

In [ ]:
gifw(append_times=2)

In [ ]:
elevations

In [ ]:
val = elevations % 30
np.stack([val[:len(val) // 2], val[len(val) // 2::-1]], axis=0)

In [ ]:
val

In [ ]:
val[len(val) // 2:][::-1]

In [ ]:
import math
math.cos(0)

In [ ]:
from scipy.interpolate import interp2d
import torch.nn.functional as F
import torch
from awesome.run.functions import get_mpl_figure, saveable
@saveable()
def plot_grid_prediction(image, 
                         res_hull_disc, 
                         fg, bg, color_fg, color_bg, color_seg, 
                         size: float = 5,
                         tight: bool = True):
    
    
    thresh = 0.1
    def subsample(img: np.ndarray):
        x = torch.arange(-1, 1, 6 / img.shape[-2])
        y = torch.arange(-1, 1, 6 / img.shape[-1])
        xx, yy = torch.meshgrid(x, y)
        flowgrid = torch.stack((yy, xx), dim=-1).float()[None,...]
        return F.grid_sample(img[None,...], flowgrid, align_corners=True)[0,...]

    subsampled_image = subsample(image).permute(1, 2, 0)
    extent = (0, subsampled_image.shape[1], subsampled_image.shape[0], 0)

    fig, ax = get_mpl_figure(1, 1, size=size, tight=tight, ratio_or_img=subsampled_image[...,0].numpy())
    
    ax.imshow(subsampled_image, extent=extent)

    ax.pcolormesh(subsampled_image, edgecolors="#a2a6ab", linewidth=.5, facecolors='none', rasterized=True, zorder=0)

    res_hull_color = np.zeros(res_hull_disc.shape[-2:] + (4,), dtype=np.float32)
    res_hull_color[..., 0:3] = color_seg

    alpha = .9
    res_hull_color[..., 3] = (1 - res_hull_disc.squeeze().numpy()) * alpha

    ax.imshow(subsample(torch.tensor(res_hull_color).permute(2, 0, 1)).permute(1, 2, 0), extent=extent)


    sub_fg = subsample(torch.tensor(fg.unsqueeze(0).float()))[0]
    sub_fg = (sub_fg > thresh).float()
    fg_pred_color = np.zeros(sub_fg.shape + (4,), dtype=np.float32)
    fg_pred_color[..., 0:3] = color_fg
    alpha_fg_scribble = .9
    fg_pred_color[..., 3] = sub_fg * alpha_fg_scribble
    ax.imshow(fg_pred_color, extent=extent)


    sub_bg = subsample(torch.tensor(bg.unsqueeze(0).float()))[0]
    sub_bg = (sub_bg > thresh).float()
    bg_pred_color = np.zeros(sub_bg.shape + (4,), dtype=np.float32)
    bg_pred_color[..., 0:3] = color_bg
    alpha_bg_scribble = .9
    bg_pred_color[..., 3] = sub_bg * alpha_bg_scribble
    ax.imshow(bg_pred_color, extent=extent)



    if not tight:
        ax.set_ylim(subsampled_image.shape[0] - 1, 0)
        ax.set_xlim(0, subsampled_image.shape[1] -1)
   
    
    # x_pos = [i._x for i in ax.get_xticklabels() if i._x <= subsampled_image.shape[1]]
    # x_lab = [round((i / subsampled_image.shape[1]), 2) for i in x_pos]
    # x_lab = ["" for x in x_lab]
    # ax.set_xticks(x_pos, x_lab)

    # y_pos = [i._y for i in ax.get_yticklabels() if i._y <= subsampled_image.shape[0]]
    # y_lab = [round((i / subsampled_image.shape[0]), 2) for i in y_pos]
    # y_lab = ["" for x in y_lab]
    # ax.set_yticks(y_pos, y_lab)

    # if tight:
    #     sizes = subsampled_image.shape[:2]
    #     dpi = 300
    #     fig.set_size_inches(15 * (sizes[1] / dpi), 15 * (sizes[0] / dpi), forward = False)
    return fig

fig = plot_grid_prediction(img, res.get("segmentation"), fg, bg, color_fg, color_bg, color_seg, 
                     save=True,
                     override=True, 
                     path="./notebooks/output/convex_teaser/teaser_explicit", 
                     tight=True,
                     ext=["png", "pdf"])
display(fig)
plt.close(fig)

In [ ]:
from scipy.interpolate import interp2d
import torch.nn.functional as F
import torch

@saveable()
def plot_grid_scribbles(image, 
                         fg, bg, color_fg, color_bg, 
                         size: float = 5,
                         tight: bool = True):
    
   
    thresh = 0.1
    def subsample(img: np.ndarray):
        x = torch.arange(-1, 1, 6 / img.shape[-2])
        y = torch.arange(-1, 1, 6 / img.shape[-1])
        xx, yy = torch.meshgrid(x, y)
        flowgrid = torch.stack((yy, xx), dim=-1).float()[None,...]
        return F.grid_sample(img[None,...], flowgrid, align_corners=True)[0,...]

    subsampled_image = subsample(image).permute(1, 2, 0)
    extent = (0, subsampled_image.shape[1], subsampled_image.shape[0], 0)

    fig = plt.figure(figsize=(size, size))

    #x_ext = sizes[1] / dpi, sizes[0] / dpi
    if tight:
        sizes = subsampled_image.shape[:2]
        dpi = 300
        fig.set_size_inches(15 * (sizes[1]-1) / dpi, 15 * (sizes[0]-1) / dpi, forward = False)
    
    ax = plt.Axes(fig, [0., 0., 1, 1])
    
    ax.set_axis_off()
    ax = fig.add_axes(ax, computed_zorder=False)
    
    ax.imshow(subsampled_image, extent=extent)

    #ax.pcolormesh(subsampled_image, edgecolors="#a2a6ab", linewidth=.5, facecolors='none',rasterized=True, zorder=0)

    sub_fg = subsample(torch.tensor(fg.unsqueeze(0).float()))[0]
    sub_fg = (sub_fg > thresh).float()
    fg_pred_color = np.zeros(sub_fg.shape + (4,), dtype=np.float32)

    fg_pred_color[..., 0:3] = color_fg
    alpha_fg_scribble = .9
    fg_pred_color[..., 3] = sub_fg * alpha_fg_scribble

    ax.imshow(fg_pred_color, extent=extent)


    sub_bg = subsample(torch.tensor(bg.unsqueeze(0).float()))[0]
    sub_bg = (sub_bg > thresh).float()
    bg_pred_color = np.zeros(sub_bg.shape + (4,), dtype=np.float32)

    bg_pred_color[..., 0:3] = color_bg
    alpha_bg_scribble = .9
    bg_pred_color[..., 3] = sub_bg * alpha_bg_scribble

    ax.imshow(bg_pred_color, extent=extent)

    ax.set_ylim(bg_pred_color.shape[0]-1, 0)
    ax.set_xlim(0, bg_pred_color.shape[1]-1,)
   
    x_pos = [i._x for i in ax.get_xticklabels() if i._x <= subsampled_image.shape[1]]
    x_lab = [round((i / subsampled_image.shape[1]), 2) for i in x_pos]
    x_lab = ["" for x in x_lab]
    ax.set_xticks(x_pos, x_lab)

    y_pos = [i._y for i in ax.get_yticklabels() if i._y <= subsampled_image.shape[0]]
    y_lab = [round((i / subsampled_image.shape[0]), 2) for i in y_pos]
    y_lab = ["" for x in y_lab]
    ax.set_yticks(y_pos, y_lab)

    return fig

fig = plot_grid_scribbles(image, fg, bg, color_fg, color_bg, 
                     save=True,
                     override=True, 
                     tight_layout=True,
                     path="./notebooks/output/teaser_image", 
                     tight=True,
                     ext=["png", "pdf"])
fig

In [ ]:
subsampled_image.shape[2]

In [ ]:

fig.get_axes()[0].get_xticklabels()
fig

In [ ]:
for lab in fig.get_axes()[0].get_xticklabels():
    idx = lab._x
    img_x = round((image.shape[2] / idx), 1)
    lab.set_text(f"{img_x}")

In [ ]:
fig

In [ ]:
sub_fg.unique()

In [ ]:
image

In [ ]:
F.grid_sample

In [ ]:
from scipy.signal import resample

plt.figure()

plt.imshow(image.permute(1, 2, 0)[::3, ::3, :])

In [ ]:
cmap

In [ ]:
to_hex(color_bg)